# Part 1

In [1]:
# Stolen from day 6 and modified for today.
import itertools

class Matrix:
    ''' A dense matrix. '''
    def __init__(self, rows, cols, initial=' '):
        self._rows = rows
        self._cols = cols
        self._cells = [[initial] * cols for _ in range(rows)]
    
    def __repr__(self):
        return 'Matrix<{}x{}>'.format(self._rows, self._cols)
    
    def __str__(self):
        render = ''
        for row in self._cells:
            render += ''.join(str(v) for v in row) + '\n'
        return render

    def __getitem__(self, key):
        row, col = key
        if row < 0 or row >= self._rows or col < 0 or col >= self._cols:
            raise KeyError(key)
        return self._cells[row][col]
    
    def __setitem__(self, key, val):
        row, col = key
        if row < 0 or row >= self._rows or col < 0 or col >= self._cols:
            raise KeyError(key)
        self._cells[row][col] = val
    
    def clone(self):
        m = Matrix(self._rows, self._cols)
        for row, col in itertools.product(range(self._rows), range(self._cols)):
            m[row,col] = self[row,col]
        return m

    def get(self, row, col, default=None):
        try:
            return self[row,col]
        except (IndexError,KeyError):
            return default
    
    @property
    def size(self):
        return self._rows, self._cols

    def values(self):
        for row, col in itertools.product(range(self._rows), range(self._cols)):
            yield self[row, col]

In [2]:
def parse_map(text):
    ''' Parse text into a map. '''
    lines = text.split('\n')
    rows = len(lines)
    cols = len(lines[0])
    mat = Matrix(rows, cols, initial='.')
    for row, line in enumerate(lines):
        for col, char in enumerate(line):
            if char != '.':
                mat[row,col] = char
    return mat

In [3]:
test_text = '''.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.'''
test_map = parse_map(test_text)
print(test_map)

.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.



In [4]:
from collections import Counter

def adjacents(map_, x,y):
    ''' Return counts of the types of the 8 adjacent cells, ignoring 
    cells that are outside the bounds. '''
    counter = Counter([
        map_.get(x-1, y-1), map_.get(x  , y-1), map_.get(x+1, y-1),
        map_.get(x-1, y  ),                     map_.get(x+1, y  ),
        map_.get(x-1, y+1), map_.get(x  , y+1), map_.get(x+1, y+1),
    ])
    del counter[None]
    return counter

In [5]:
adjacents(test_map, 0, 0)

Counter({'.': 2, '#': 1})

In [6]:
adjacents(test_map, 0, 0)

Counter({'.': 2, '#': 1})

In [7]:
def play_round(map_):
    ''' Play one round and return an updated copy of map. '''
    map2 = map_.clone()
    rows, cols = map_.size
    for x, y in itertools.product(range(rows), range(cols)):
        cell = map_[x, y]
        adj = adjacents(map_, x, y)
        if cell == '.' and adj['|'] >= 3:
            map2[x,y] = '|'
        elif cell == '|' and adj['#'] >= 3:
            map2[x,y] = '#'
        elif cell == '#' and not (adj['#'] >= 1 and adj['|'] >= 1):
            map2[x,y] = '.'
    return map2

In [8]:
test_text = '''.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.'''
test_map = parse_map(test_text)
n = 0

In [9]:
# Run this cell multiple times
test_map = play_round(test_map)
print(test_map)
n += 1
print(n)

.......##.
......|###
.|..|...#.
..|#||...#
..##||.|#|
...#||||..
||...|||..
|||||.||.|
||||||||||
....||..|.

1


In [10]:
def map_value(map_):
    counter = Counter(cell for cell in map_.values())
    return counter['|'] * counter['#']

In [11]:
map_value(test_map)

480

In [12]:
with open('input.txt') as input_:
    map_ = parse_map(input_.read().strip())

In [13]:
print(map_)

.|#..|#.|.#.#..|.#..##......#...#..#...|.#.#|.|..#
.|||.##.#|#..|#.|#.|.............|.....|.....|#||.
|...#.||.#.|.|#.#....##..||#...#|..|#...##...|#...
...|.##|...|.||...##.#.##.|...#.|#..|..........#.#
#..#....#.#.|....#...#|#....|.###........|#.....#.
#.##.#..#|##.|||.|..|.|#.....|#.....|||||.#.#.|#..
..|....#|...#...#|##...#...|.|...#.#.||.|.|..#.|#.
..##.|..||...|.#..#.|.#..|..#.###.#....###........
#.....#|.....#.|#....#.|.||||.##..||.#.|.#|.....#|
....##...#.......#..|..#||...#||#.|.|.||..#.|||##.
###.##..||..###.#..#.#.|.....#.|#..#.#|...|#..|.#.
...#|###|||.||...#.||..##..|#|...||#....#...||.#..
|..####..#.....|..#..||.#.....##|....||..|......#|
|##...|.#......#||#|......#|#.|#....|..#||....|.||
...#..||#.||..#|.###|.#.|...|...|##|...##....|.||#
.||.|####.#|..|#.....|..#.#.|#.....|.|...##..|....
|.#.........#|....|....#||........#.#.....#....|||
...|#|..#|...|.|..#.##|#..|......#.....|.||..#||#|
.#.....||.#..##.|....#...|.|.#...|.#...|#|..|#|#..
.#.#|.|#.#.|.#.#.||.....|#..|.#

In [14]:
for _ in range(10):
    map_ = play_round(map_)
map_value(map_)

514944

In [15]:
print(map_)

|||||||........#|||....||||||||...##|||||||||||||.
||||||||......###|||..|||||||||...##||||###|||#|||
||||||||........#.....||||||#|||...##|######||#||.
||||||||.............|||||####||.#######..##|||||.
||||||||.............|||||##.#.||######...##||||..
|||||||..............||||##.....|#####....||||##..
|||||###...............#####.....##......||||##...
|||||##....###..........####.............|||###...
.|||##.....#####........####..............####....
.||||##....#####.........###..............##......
|||||##....##|||..................................
||||||##..###|||........###...............####....
|||||||#######||........###.............######....
|||||||||#####..........###.........######||##....
|||||##||||##....##.........####....##########....
|||||##|||##....###.#####...####||.##||######.....
||||||||||##......######|..###|||||||||##.........
||||||||||####....###||||..##|||||||||###.........
|||||||||||###|...##|||##..##|||||||||##..........
|||#||||||||||##...######..##||

# Part 2

In [17]:
with open('input.txt') as input_:
    map_ = parse_map(input_.read().strip())
last = 0
val = 0
i = 0

In [24]:
# Run this cell repeatedly until map seems to converge
for _ in range(100):
    i += 1
    map_ = play_round(map_)
    last, val = val, map_value(map_)
    print('round={} val={} delta={}'.format(i, val, val-last))

round=601 val=184034 delta=3962
round=602 val=186930 delta=2896
round=603 val=190619 delta=3689
round=604 val=193890 delta=3271
round=605 val=195300 delta=1410
round=606 val=195300 delta=0
round=607 val=194856 delta=-444
round=608 val=193050 delta=-1806
round=609 val=192765 delta=-285
round=610 val=191700 delta=-1065
round=611 val=188682 delta=-3018
round=612 val=186017 delta=-2665
round=613 val=182320 delta=-3697
round=614 val=180687 delta=-1633
round=615 val=180348 delta=-339
round=616 val=179820 delta=-528
round=617 val=181440 delta=1620
round=618 val=182910 delta=1470
round=619 val=185433 delta=2523
round=620 val=188020 delta=2587
round=621 val=189405 delta=1385
round=622 val=190704 delta=1299
round=623 val=189660 delta=-1044
round=624 val=187074 delta=-2586
round=625 val=186186 delta=-888
round=626 val=187000 delta=814
round=627 val=186238 delta=-762
round=628 val=187152 delta=914
round=629 val=190494 delta=3342
round=630 val=193328 delta=2834
round=631 val=195789 delta=2461
round

After playing a few hundred rounds, it seems like it settles into a pattern. Notice how this sequence goes through the same 28 values twice.

    round=611 val=188682 delta=-3018
    round=612 val=186017 delta=-2665
    round=613 val=182320 delta=-3697
    round=614 val=180687 delta=-1633
    round=615 val=180348 delta=-339
    round=616 val=179820 delta=-528
    round=617 val=181440 delta=1620
    round=618 val=182910 delta=1470
    round=619 val=185433 delta=2523
    round=620 val=188020 delta=2587
    round=621 val=189405 delta=1385
    round=622 val=190704 delta=1299
    round=623 val=189660 delta=-1044
    round=624 val=187074 delta=-2586
    round=625 val=186186 delta=-888
    round=626 val=187000 delta=814
    round=627 val=186238 delta=-762
    round=628 val=187152 delta=914
    round=629 val=190494 delta=3342
    round=630 val=193328 delta=2834
    round=631 val=195789 delta=2461
    round=632 val=197050 delta=1261
    round=633 val=197532 delta=482
    round=634 val=195300 delta=-2232
    round=635 val=194856 delta=-444
    round=636 val=193050 delta=-1806
    round=637 val=192765 delta=-285
    round=638 val=191700 delta=-1065

    round=639 val=188682 delta=-3018
    round=640 val=186017 delta=-2665
    round=641 val=182320 delta=-3697
    round=642 val=180687 delta=-1633
    round=643 val=180348 delta=-339
    round=644 val=179820 delta=-528
    round=645 val=181440 delta=1620
    round=646 val=182910 delta=1470
    round=647 val=185433 delta=2523
    round=648 val=188020 delta=2587
    round=649 val=189405 delta=1385
    round=650 val=190704 delta=1299
    round=651 val=189660 delta=-1044
    round=652 val=187074 delta=-2586
    round=653 val=186186 delta=-888
    round=654 val=187000 delta=814
    round=655 val=186238 delta=-762
    round=656 val=187152 delta=914
    round=657 val=190494 delta=3342
    round=658 val=193328 delta=2834
    round=659 val=195789 delta=2461
    round=660 val=197050 delta=1261
    round=661 val=197532 delta=482
    round=662 val=195300 delta=-2232
    round=663 val=194856 delta=-444
    round=664 val=193050 delta=-1806
    round=665 val=192765 delta=-285
    round=666 val=191700 delta=-1065

So it seems like I can predict round 1e9 using modular arithmetic.

In [25]:
def compute_value(age):    
    
    pattern = [188682,186017,182320,180687,180348,179820,181440,182910,185433,188020,
               189405,190704,189660,187074,186186,187000,186238,187152,190494,193328,
               195789,197050,197532,195300,194856,193050,192765,191700]
    starts_at = 611
    idx = (age - starts_at) % len(pattern)
    return pattern[idx]

In [27]:
# Compare these to the output from above.
assert compute_value(611) == 188682
assert compute_value(635) == 194856
assert compute_value(650) == 190704
assert compute_value(665) == 192765

In [28]:
compute_value(1_000_000_000)

193050

In [29]:
# The map looks purdy.
print(map_)

...........#||||.........#||||.........#||||......
...........##|||.........##|||.........##|||......
...........##||||........##||||........##||||.....
............##|||.........##|||.........##|||.....
...........##||||.........##||||........##||||....
.........####|||...........##|||.........##|||....
.......||###||||..........##|||.........##|||.....
.......|||||||||||........##|||.........##|||.....
.......|||||||||||#......##|||.........##|||......
........|||||..||##......##|||.........##|||......
..........|....|||##...###|||.........##|||.......
................||#######||||.........##|||.......
................||||###|||||.........##|||........
.................||||||||||..........##|||........
##................|||||||...........##|||.........
##..................|||.............##|||.........
|###...............................##|||..........
||####.............................##|||..........
||||####..........................##|||.........##
||||||####.....................